In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('data.csv', nrows=400)

In [4]:
df.columns

Index(['# Timestamp', 'Type of mobile', 'MMSI', 'Latitude', 'Longitude',
       'Navigational status', 'ROT', 'SOG', 'COG', 'Heading', 'IMO',
       'Callsign', 'Name', 'Ship type', 'Cargo type', 'Width', 'Length',
       'Type of position fixing device', 'Draught', 'Destination', 'ETA',
       'Data source type', 'A', 'B', 'C', 'D'],
      dtype='object')

In [69]:
df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))
df

,#_timestamp,type_of_mobile,mmsi,latitude,longitude,navigational_status,rot,sog,cog,heading,...,length,type_of_position_fixing_device,draught,destination,eta,data_source_type,a,b,c,d
0,01/05/2023 00:00:00,Base Station,2190064,56.7166,11.51910,Unknown value,NaN,NaN,NaN,NaN,...,NaN,GPS,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
1,01/05/2023 00:00:00,Class A,250006549,57.6578,11.01110,Under way using engine,0.0,12.0,135.5,132.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
2,01/05/2023 00:00:00,Class A,219340000,56.6968,8.21854,Under way using engine,NaN,0.0,300.4,NaN,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
3,01/05/2023 00:00:00,Class A,265012560,55.4109,12.93320,Under way using engine,0.0,0.0,265.4,337.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
4,01/05/2023 00:00:00,Class A,219019887,55.8648,10.54880,Under way using engine,0.0,0.0,8.0,8.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,01/05/2023 00:00:02,Class A,231807000,57.6874,11.89450,Under way using engine,0.0,0.0,275.4,125.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
396,01/05/2023 00:00:02,Class A,219030053,57.0582,9.90081,Unknown value,NaN,0.0,351.6,NaN,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
397,01/05/2023 00:00:02,Base Station,2190065,55.6568,8.67140,Unknown value,NaN,NaN,NaN,NaN,...,NaN,Surveyed,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
398,01/05/2023 00:00:02,Class A,219026071,56.7049,8.22219,Under way using engine,0.0,0.0,234.9,249.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN


## Connecting to the database

In [80]:
from pymongo import MongoClient
import numpy as np
import multiprocessing as mp

In [61]:
mongos1 = "mongodb://localhost:27019/"
mongos2 = "mongodb://localhost:27020/"

In [62]:
client = MongoClient(mongos1)
db = client["ships"]
collection = db["ship_positions"]

In [63]:
data = df.to_dict(orient='records')

In [85]:
def insert_data(indices, n):
    print("chunk", n, "processing")
    data = df.iloc[indices]
    insert = data.to_dict(orient='records')
    result = collection.insert_many(insert)
    print("inserted ids:", result)
    
def parallelize_inserts(indices, n_chunks, n_cpus):
    pool = mp.Pool(n_cpus)
    chunks = np.array_split(indices, n_chunks)
    temp = pool.starmap(insert_data, [(chunk,n_cpus) for chunk in chunks])


In [86]:
parallelize_inserts(indices, 20, 10)

Process SpawnPoolWorker-81:
Process SpawnPoolWorker-82:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'insert_data' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/V

Process SpawnPoolWorker-92:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 368, in get
    return _ForkingPickler.loads(res)
AttributeError: Can't get attribute 'insert_data' on <module '__main__' (built-in)>
Process SpawnPoolWorker-93:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/V

KeyboardInterrupt: 

n
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Process SpawnPoolWorker-68:
Process SpawnPoolWorker-70:
Process SpawnPoolWorker-79:
Process SpawnPoolWorker-63:
Process SpawnPoolWorker-62:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File 

In [70]:
cpus_max = mp.cpu_count()

In [71]:
cpus_max

10

In [72]:
indices = range(len(df))
indices

range(0, 400)

In [10]:
chunksize=500
chunkId=10

cols = pd.read_csv('data.csv', nrows=0).columns
df = pd.read_csv('data.csv', skiprows=chunksize * chunkId, nrows=25000, names=cols)
df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))
insert = df.to_dict(orient='records')

for _id, row in enumerate(insert):
    row['id'] = chunkId * chunksize + _id

# print(row)

In [13]:
df

,#_timestamp,type_of_mobile,mmsi,latitude,longitude,navigational_status,rot,sog,cog,heading,...,length,type_of_position_fixing_device,draught,destination,eta,data_source_type,a,b,c,d
0,01/05/2023 00:00:39,Class A,311001084,56.6748,7.49712,Under way using engine,-0.4,16.3,207.0,202.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
1,01/05/2023 00:00:39,Class A,219003217,55.0938,10.23930,Under way using engine,0.0,0.2,264.9,107.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
2,01/05/2023 00:00:39,Class A,266343000,55.7427,10.79790,Under way using engine,0.0,16.0,190.1,190.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
3,01/05/2023 00:00:39,Class A,210529000,57.6720,8.64983,Under way using engine,0.0,10.8,258.0,254.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
4,01/05/2023 00:00:39,Class A,209869000,55.3706,13.14880,Under way using engine,0.0,0.0,259.1,313.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,01/05/2023 00:03:56,Base Station,2190065,55.6568,8.67140,Unknown value,NaN,NaN,NaN,NaN,...,NaN,Surveyed,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
24996,01/05/2023 00:03:56,Class A,352002487,57.0476,11.73160,Under way using engine,0.0,14.1,342.3,343.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
24997,01/05/2023 00:03:56,Class A,311000795,56.3848,11.47990,Under way using engine,0.0,17.7,204.7,202.0,...,150.0,GPS,7.9,EE TLL,02/05/2023 14:00:00,AIS,117.0,33.0,7.0,15.0
24998,01/05/2023 00:03:56,Class A,219000775,56.7016,8.22098,Under way using engine,0.0,0.1,145.8,223.0,...,NaN,Undefined,NaN,NaN,NaN,AIS,NaN,NaN,NaN,NaN
